In [1]:
import numpy as np
import pandas as pd
import time
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib
import os

%load_ext autoreload
%autoreload 2
# %matplotlib notebook
%matplotlib inline

In [2]:
# body_part = "head"
# file_names = ['./training_data/head_data_1.log',
#              './training_data/head_data_2.log',
#              './training_data/head_data_3.log',
#              './training_data/head_data_4.log',
#              './training_data/head_data_5.log',
#              './training_data/head_data_6.log']

# body_part = "shoulder_right"
# file_names = [
#              './training_data_old/shoulder_right_data_1.log',
#              './training_data_old/shoulder_right_data_2.log',
#              './training_data_old/shoulder_right_data_3.log',
#              './training_data_old/shoulder_right_data_4.log',
#              './training_data_old/shoulder_right_data_5.log']

# body_part = "shoulder_right"
# file_names = [
#              './training_data/shoulder_right_train_10_17.log',
#              './training_data_old/shoulder_right_data_6.log',
#              './training_data_old/shoulder_right_data_7.log',
#              './training_data_old/shoulder_right_data_8.log',
#              './training_data_old/shoulder_right_data_9.log',
#             ]

body_part = "shoulder_left"
file_names = ['./training_data/shoulder_left_train_5_6_7_9_10.log']

# file_names = ['./training_data/shoulder_left_5.log',
#               './training_data/shoulder_left_6.log',
#               './training_data/shoulder_left_7.log']

# body_part = "shoulder_left"
# file_names = ['./training_data/shoulder_left_1.log',
#               './training_data/shoulder_left_2.log',
#               './training_data/shoulder_left_3.log']

In [3]:
orig_dataset = pd.concat([pd.read_csv(f, delim_whitespace=True, header=0) for f in file_names])
orig_dataset = orig_dataset.values[1:len(orig_dataset)-1,0:]

abnormal_threshold = 0.5
dataset = []

for name in file_names:
    df = pd.read_csv(name, delim_whitespace=True, header=0)
    
    # interpolate nan outputs (loss tracking)
#     for c in df.columns[-3:]:
#         df[c] = df[c].interpolate()
    
    # interpolate abnormal inputs
    for c in df.columns[:-3]:
        bad_idx = df.index[df[c].pct_change().abs().ge(abnormal_threshold)]
        df.loc[bad_idx, c] = np.nan
        df[c] = df[c].interpolate()
    
    dataset.append(df)
    
dataset_len = [len(data) for data in dataset]
dataset = pd.concat(dataset) 

print(f'{np.sum(dataset_len)} values')

dataset = dataset.values[1:len(dataset)-1,0:]
dataset = dataset[abs(dataset[:,12])!=0.0,:]
dataset = dataset[abs(dataset[:,13])!=0.0,:]
dataset = dataset[abs(dataset[:,14])!=0.0,:]

print(f'{len(dataset)} values after filtering outliers')

euler_set = dataset[:, 12:15]
sensors_set = dataset[:, :12]
orig_sensors_set = orig_dataset[:, :12]
print(f'max euler {str(np.amax(euler_set))}')
print(f'min euler {str(np.amin(euler_set))}')

250240 values
250238 values after filtering outliers
max euler nan
min euler nan


In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from libs.orientation_utils import compute_rotation_matrix_from_euler, compute_ortho6d_from_rotation_matrix, compute_rotation_matrix_from_ortho6d, compute_euler_angles_from_rotation_matrices

In [5]:
euler_set_in = np.zeros_like(euler_set)
euler_set_in[:, 0] = euler_set[:, 2] 
euler_set_in[:, 1] = euler_set[:, 1] 
euler_set_in[:, 2] = euler_set[:, 0] 

euler_set = torch.Tensor(euler_set_in).cuda()
rot_set = compute_rotation_matrix_from_euler(euler_set)
rot_set = compute_ortho6d_from_rotation_matrix(rot_set).cpu().detach().numpy()

In [6]:
model_path = f"./outputs/{body_part}_dvbf_ad_rot6D"

sensors_scaler = joblib.load(f'{model_path}/scaler.pkl')

# Split magnetic sensors into 4 independent distributions again
orig_sensors_set = sensors_scaler.transform(orig_sensors_set).reshape(-1, 4, 3)
sensors_set = sensors_scaler.transform(sensors_set).reshape(-1, 4, 3)

In [7]:
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from libs.data_modules import SensorsDataModule
from libs.nn_models import mse_seq
from libs.dvbf_models import DVBF

In [8]:
model = DVBF.load_from_checkpoint(checkpoint_path=f"{model_path}/best.ckpt")

/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
device = "cuda"
model.to(device)
torch.set_grad_enabled(False)
model.eval()

DVBF(
  (initial_lstm): LSTM(12, 100, batch_first=True, dropout=0.1, bidirectional=True)
  (initial_to_params): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=32, bias=True)
  )
  (w1_to_z1): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=16, bias=True)
  )
  (matrix_params): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=16, bias=True)
    (3): Softmax(dim=None)
  )
  (decoder_model): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=24, bias=True)
  )
  (regressor_model): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=12, bias=True)
  )
  (encoder_models

In [10]:
def plot_test(sensors_set, euler_set, look_back=5, ab_idx=None, save_name=None):

    # Specific testing for abnormalities
    
    if body_part == "shoulder_right":
        start_idx = 110090
        end_idx = 110790
        ab_idx = 110690
    elif body_part == "shoulder_left":
#         start_idx = 21500 # 4500
#         end_idx = 22880 # 4850
        start_idx = 20500 # 4500
        end_idx = 23880 # 4850
        ab_idx = 22626
    elif body_part == "head":
        start_idx = 900
        end_idx = 950

    test_idxs = slice(start_idx, end_idx)

    # Prepare inputs and do the prediction
    in_set = []
    for i in range(start_idx, end_idx):
        in_set.append(sensors_set[i:i+look_back])

    in_set = torch.tensor(sensors_set[start_idx:end_idx][None], dtype=torch.float32).to(device)
    out_set = model.predict(in_set)[0]#[:, -1]
    rot_pred = compute_rotation_matrix_from_ortho6d(out_set)

    euler_pred = compute_euler_angles_from_rotation_matrices(rot_pred)
    euler_gt = torch.tensor(euler_set[test_idxs], dtype=torch.float32).to(device)
    
    print(f"MSE: {mse_seq(euler_pred, euler_gt)}")
    
    euler_pred = euler_pred.cpu().detach().numpy()
    euler_gt = euler_gt.cpu().detach().numpy()

    # Plotting
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.scatter(euler_gt[:, 0], euler_gt[:, 1], euler_gt[:, 2], c=np.arange(len(euler_gt)), alpha=0.1)
    sc = ax.plot(euler_pred[:,0], euler_pred[:,1], euler_pred[:,2])
    sc = ax.scatter(euler_pred[:,0], euler_pred[:,1], euler_pred[:,2], c=np.arange(len(euler_pred)), alpha=0.5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    plt.colorbar(sc)
    plt.show()
    
    ax.set_xlim([-2, 0.5])
    ax.set_ylim([-1.5, 0.])
    ax.set_zlim([-2, 0.])
    
    if save_name:
        fig.savefig('./outputs/'+body_part+'_'+save_name+'.png', dpi=100, bbox_inches='tight')
    
    # Abnormal analysis
    latent_dist = None
    if ab_idx is not None:
        ab_idx = ab_idx - start_idx
        latent_dist = model.inv_meas(in_set[:, ab_idx])
        print(f"Abnormal input: {in_set[:, ab_idx]}")
    return latent_dist

In [11]:
test_euler_set = dataset[:, 12:15]

In [12]:
# Plot with raw inputs
test_latent_dist = plot_test(sensors_set, test_euler_set, look_back=10)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Plot with interpolated inputs
orig_latent_dist = plot_test(orig_sensors_set, test_euler_set, save_name="dvbf")

In [ ]:
test_latent_mean = torch.stack(test_latent_dist[0])
test_latent_var = torch.stack(test_latent_dist[1])

In [ ]:
orig_latent_mean = torch.stack(orig_latent_dist[0])
orig_latent_var = torch.stack(orig_latent_dist[1])

In [ ]:
(test_latent_mean - orig_latent_mean).abs().cpu().numpy()

In [ ]:
(test_latent_var - orig_latent_var).abs().cpu().numpy()

In [ ]:
test_latent_var

In [ ]:
orig_latent_var

In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
abnormal_idx = np.where(np.abs(test_transformed_sensors_set - orig_transformed_sensors_set) > 0.5)

In [ ]:
abnormal_idx

In [ ]:
test_sensors_set[abnormal_idx]

In [ ]:
orig_sensors_set[abnormal_idx]